In [1]:
import requests

URL = "https://api.hyperliquid.xyz/info"

payload = {
    "type": "allMids"
}

resp = requests.post(URL, json=payload, timeout=10)
resp.raise_for_status()

mids = resp.json()

print(f"BTC mid price: {mids.get('BTC')}")
print(f"ETH mid price: {mids.get('ETH')}")


BTC mid price: 88337.5
ETH mid price: 2939.95


### Fetch prices

In [2]:
import requests

def fetch_mids():
    url = "https://api.hyperliquid.xyz/info"
    payload = {"type": "allMids"}
    r = requests.post(url, json=payload)
    r.raise_for_status()
    return r.json()


### Maintain rolling price history

In [3]:
from collections import deque

price_history = {
    "BTC": deque(maxlen=50),
    "ETH": deque(maxlen=50)
}


### Update Prices

In [4]:
mids = fetch_mids()

price_history["BTC"].append(float(mids["BTC"]))
price_history["ETH"].append(float(mids["ETH"]))


### Compute SMA

In [5]:
import numpy as np

def compute_sma(prices):
    if len(prices) == 0:
        return None
    return np.mean(prices)


### Signal

In [6]:
def generate_signal(price, sma):
    if sma is None:
        return 0
    return 1 if price > sma else 0


### Compute signals for BTC & ETH

In [7]:
signals = {}

for asset in ["BTC", "ETH"]:
    price = price_history[asset][-1]
    sma = compute_sma(price_history[asset])
    signals[asset] = generate_signal(price, sma)

signals


{'BTC': 0, 'ETH': 0}

### Trade Engine

In [8]:
state = {
    "BTC": {"position": 0, "entry_price": None, "pnl": 0.0, "trades": 0},
    "ETH": {"position": 0, "entry_price": None, "pnl": 0.0, "trades": 0},
}


In [9]:
def execute_trade(asset, target_pos, price, state, max_position_usd):
    current_pos = state[asset]["position"]
    
    if current_pos == target_pos:
        return None  # no trade
    
    # position size in units
    size = max_position_usd / price
    
    trade = {
        "asset": asset,
        "price": price,
        "direction": "BUY" if target_pos == 1 else "SELL",
        "size": size
    }
    
    # Enter long
    if current_pos == 0 and target_pos == 1:
        state[asset]["position"] = 1
        state[asset]["entry_price"] = price
        state[asset]["trades"] += 1
    
    # Exit long
    elif current_pos == 1 and target_pos == 0:
        entry = state[asset]["entry_price"]
        pnl = (price - entry) * size
        state[asset]["pnl"] += pnl
        state[asset]["position"] = 0
        state[asset]["entry_price"] = None
        state[asset]["trades"] += 1
        trade["realized_pnl"] = pnl
    
    return trade


### Run Execution

In [10]:
trades = []

limits = {"BTC": 5000, "ETH": 3000}

for asset in ["BTC", "ETH"]:
    trade = execute_trade(
        asset=asset,
        target_pos=signals[asset],
        price=price_history[asset][-1],
        state=state,
        max_position_usd=limits[asset]
    )
    if trade:
        trades.append(trade)

trades


[]

### Logging

In [11]:
import pandas as pd
from datetime import datetime

log_rows = []

def log_trade(trade, signal, position):
    row = {
        "timestamp": datetime.utcnow().isoformat(),
        "asset": trade["asset"],
        "price": trade["price"],
        "signal": signal,
        "direction": trade["direction"],
        "size": trade["size"],
        "position": position,
        "realized_pnl": trade.get("realized_pnl", 0.0),
    }
    log_rows.append(row)


In [12]:
from datetime import datetime
import time

def status_print(asset, price, sma, signal, position, trade=None):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    
    sma_str = f"{sma:.2f}" if sma is not None else "NA"
    
    msg = (
        f"[{ts}] {asset} | "
        f"Price: {price:.2f} | "
        f"SMA: {sma_str} | "
        f"Signal: {'LONG' if signal == 1 else 'FLAT'} | "
        f"Position: {position}"
    )
    
    if trade:
        msg += f" | TRADE: {trade['direction']} {trade['size']:.4f}"
    
    print(msg)


### Log executed trades

In [13]:
for trade in trades:
    asset = trade["asset"]
    log_trade(
        trade=trade,
        signal=signals[asset],
        position=state[asset]["position"]
    )

pd.DataFrame(log_rows)


""


### Wrap in a loop (scheduler)

In [14]:
for _ in range(10):
    mids = fetch_mids()
    
    for asset in ["BTC", "ETH"]:
        price_history[asset].append(float(mids[asset]))
    
    for asset in ["BTC", "ETH"]:
        price = price_history[asset][-1]
        sma = compute_sma(price_history[asset])
        signal = generate_signal(price, sma)
        
        trade = execute_trade(
            asset, signal, price, state, limits[asset]
        )
        
        status_print(
            asset=asset,
            price=price,
            sma=sma,
            signal=signal,
            position=state[asset]["position"],
            trade=trade
        )
        
        if trade:
            log_trade(trade, signal, state[asset]["position"])
    
    print("-" * 80)
    time.sleep(60)


[15:47:59] BTC | Price: 88337.50 | SMA: 88337.50 | Signal: FLAT | Position: 0
[15:47:59] ETH | Price: 2939.05 | SMA: 2939.50 | Signal: FLAT | Position: 0
--------------------------------------------------------------------------------
[15:48:59] BTC | Price: 88366.50 | SMA: 88347.17 | Signal: LONG | Position: 1 | TRADE: BUY 0.0566
[15:48:59] ETH | Price: 2942.85 | SMA: 2940.62 | Signal: LONG | Position: 1 | TRADE: BUY 1.0194
--------------------------------------------------------------------------------
[15:49:59] BTC | Price: 88284.50 | SMA: 88331.50 | Signal: FLAT | Position: 0 | TRADE: SELL 0.0566
[15:49:59] ETH | Price: 2939.95 | SMA: 2940.45 | Signal: FLAT | Position: 0 | TRADE: SELL 1.0204
--------------------------------------------------------------------------------
[15:51:00] BTC | Price: 87853.00 | SMA: 88235.80 | Signal: FLAT | Position: 0
[15:51:00] ETH | Price: 2920.35 | SMA: 2936.43 | Signal: FLAT | Position: 0
-----------------------------------------------------------

## Robust 

In [56]:
import time
import requests
import numpy as np
import pandas as pd
from collections import deque
from datetime import datetime

In [57]:
ASSETS = {
"BTC": {
"max_position_usd": 5000
},
"ETH": {
"max_position_usd": 3000
}
}

SMA_WINDOW = 100
EMA_WINDOW = 100
VOL_WINDOW = 50
VOL_RSI_WINDOW = 14
VOLUME_PERCENTILE = 70

TAKE_PROFIT_PCT = 0.003 # 0.30%
STOP_LOSS_PCT = 0.002 # 0.20%

MAX_TRADES_PER_ASSET = 10
MAX_DRAWDOWN_PCT = 2.0
STARTING_CAPITAL = 10000

LOOP_INTERVAL = 60 # seconds

In [58]:

price_history = {
    "BTC": deque(maxlen=SMA_WINDOW),
    "ETH": deque(maxlen=SMA_WINDOW)
}

vol_history = {
    "BTC": deque(maxlen=VOL_WINDOW),
    "ETH": deque(maxlen=VOL_WINDOW)
}

volume_history = {
    "BTC": deque(maxlen=100),
    "ETH": deque(maxlen=100)
}

state = {
    "BTC": {"position": 0, "entry_price": None, "pnl": 0.0, "trades": 0},
    "ETH": {"position": 0, "entry_price": None, "pnl": 0.0, "trades": 0}
}

log_rows = []


def fetch_mids():
    url = "https://api.hyperliquid.xyz/info"
    payload = {"type": "allMids"}
    r = requests.post(url, json=payload, timeout=10)
    r.raise_for_status()
    return r.json()


def fetch_orderbook(asset):
    url = "https://api.hyperliquid.xyz/info"
    payload = {
        "type": "l2Book",
        "coin": asset
    }
    r = requests.post(url, json=payload, timeout=10)
    r.raise_for_status()
    return r.json()


def fetch_recent_volume(asset, lookback_seconds=60):
    url = "https://api.hyperliquid.xyz/info"
    payload = {
        "type": "recentTrades",
        "coin": asset
    }
    r = requests.post(url, json=payload, timeout=10)
    r.raise_for_status()
    trades = r.json()

    now = time.time() * 1000  # ms
    vol = 0.0
    for t in trades:
        if now - t["time"] <= lookback_seconds * 1000:
            vol += float(t["sz"])
    return vol


In [ ]:


## =========================
## INDICATORS
## =========================

def compute_sma(prices):
    return np.mean(prices) if len(prices) > 0 else None

def compute_ema(prices, window=100):
    if len(prices) == 0:
        return None
    alpha = 2 / (window + 1)
    ema = prices[0]
    for p in prices[1:]:
        ema = alpha * p + (1 - alpha) * ema
    return ema

def compute_volatility(prices):
    if len(prices) < 2:
        return None
    returns = np.diff(prices) / prices[:-1]
    return np.std(returns)

def compute_rsi(series, window=14):
    if len(series) < window + 1:
        return None
    deltas = np.diff(series)
    gains = np.maximum(deltas, 0)
    losses = np.maximum(-deltas, 0)
    avg_gain = np.mean(gains[-window:])
    avg_loss = np.mean(losses[-window:])
    if avg_loss == 0:
        return 100.0
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

def volume_filter(current_vol, history, percentile=70):
    if len(history) < 10:
        return False
    threshold = np.percentile(history, percentile)
    return current_vol > threshold


In [60]:
def generate_signal(price, sma, ema, vol_rsi, vol_ok):
    trend_ok = price > sma and price > ema
    volatility_ok = vol_rsi is not None and vol_rsi > 70
    return 1 if (trend_ok and volatility_ok and vol_ok) else 0

In [61]:
def estimate_execution_price(orderbook, side, order_size):
    """
    side: 'BUY' or 'SELL'
    order_size: base asset size
    """
    levels = orderbook["levels"]
    
    # Hyperliquid: asks are positive px for BUY, bids for SELL
    book = levels["asks"] if side == "BUY" else levels["bids"]

    remaining = order_size
    cost = 0.0
    filled = 0.0

    for lvl in book:
        price = float(lvl["px"])
        size = float(lvl["sz"])

        take = min(size, remaining)
        cost += take * price
        filled += take
        remaining -= take

        if remaining <= 0:
            break

    if filled == 0:
        return None

    return cost / filled


def execute_trade(asset, target_pos, mid_price, state, max_position_usd):
    current_pos = state[asset]["position"]
    size = max_position_usd / mid_price

    orderbook = fetch_orderbook(asset)

    # ENTRY
    if current_pos == 0 and target_pos == 1:
        exec_price = estimate_execution_price(orderbook, "BUY", size)
        if exec_price is None:
            return None

        state[asset]["position"] = 1
        state[asset]["entry_price"] = exec_price
        state[asset]["trades"] += 1

        return {
            "asset": asset,
            "mid_price": mid_price,
            "price": exec_price,
            "direction": "BUY",
            "size": size,
            "slippage": exec_price - mid_price
        }

    # EXIT
    if current_pos == 1:
        entry = state[asset]["entry_price"]
        pnl_pct = (mid_price - entry) / entry

        exit_reason = None
        if pnl_pct >= TAKE_PROFIT_PCT:
            exit_reason = "TAKE_PROFIT"
        elif pnl_pct <= -STOP_LOSS_PCT:
            exit_reason = "STOP_LOSS"
        elif target_pos == 0:
            exit_reason = "TREND_EXIT"

        if exit_reason:
            exec_price = estimate_execution_price(orderbook, "SELL", size)
            if exec_price is None:
                return None

            pnl = (exec_price - entry) * size
            state[asset]["pnl"] += pnl
            state[asset]["position"] = 0
            state[asset]["entry_price"] = None
            state[asset]["trades"] += 1

            return {
                "asset": asset,
                "mid_price": mid_price,
                "price": exec_price,
                "direction": "SELL",
                "size": size,
                "realized_pnl": pnl,
                "exit_reason": exit_reason,
                "slippage": exec_price - mid_price
            }

    return None


In [62]:
def log_trade(trade, signal, position):
    log_rows.append({
        "timestamp": datetime.utcnow().isoformat(),
        "asset": trade["asset"],
        "mid_price": trade["mid_price"],
        "exec_price": trade["price"],
        "slippage": trade.get("slippage", 0.0),
        "signal": signal,
        "direction": trade["direction"],
        "size": trade["size"],
        "position": position,
        "realized_pnl": trade.get("realized_pnl", 0.0),
        "exit_reason": trade.get("exit_reason", "")
    })


def status_print(asset, price, sma, ema, vol_rsi, volume, signal, position, trade=None):
    ts = datetime.utcnow().strftime("%H:%M:%S")

    sma_str = f"{sma:.2f}" if sma is not None else "NA"
    ema_str = f"{ema:.2f}" if ema is not None else "NA"
    vol_rsi_str = f"{vol_rsi:.1f}" if vol_rsi is not None else "NA"
    vol_str = f"{volume:.2f}" if volume is not None else "NA"

    msg = (
        f"[{ts}] {asset} | "
        f"Mid: {price:.2f} | "
        f"SMA: {sma_str} | EMA: {ema_str} | "
        f"Vol_RSI: {vol_rsi_str} | "
        f"Vol: {vol_str} | "
        f"Signal: {'LONG' if signal else 'FLAT'} | "
        f"Position: {position}"
    )

    if trade:
        exec_px = trade.get("price")
        slip = trade.get("slippage", 0.0)
        reason = trade.get("exit_reason", "")

        msg += (
            f" | TRADE: {trade['direction']} "
            f"@ {exec_px:.2f} "
            f"(slip {slip:.2f})"
        )

        if reason:
            msg += f" [{reason}]"

    print(msg)



In [63]:



## =========================
## RISK CONTROL
## =========================

def drawdown_exceeded(state):
    total_pnl = sum(v["pnl"] for v in state.values())
    return total_pnl < -STARTING_CAPITAL * MAX_DRAWDOWN_PCT / 100


In [64]:
while True:
    mids = fetch_mids()

    for asset in ASSETS:
        price = float(mids[asset])
        price_history[asset].append(price)

        vol = compute_volatility(list(price_history[asset]))
        if vol is not None:
            vol_history[asset].append(vol)

        real_vol = fetch_recent_volume(asset)
        volume_history[asset].append(real_vol)


        sma = compute_sma(price_history[asset])
        ema = compute_ema(list(price_history[asset]))
        vol_rsi = compute_rsi(list(vol_history[asset]))
        vol_ok = volume_filter(volume_history[asset][-1], volume_history[asset])

        signal = generate_signal(price, sma, ema, vol_rsi, vol_ok)

        trade = execute_trade(
            asset, signal, price, state, ASSETS[asset]["max_position_usd"]
        )

        status_print(
            asset=asset,
            price=price,
            sma=sma,
            ema=ema,
            vol_rsi=vol_rsi,
            volume=real_vol,
            signal=signal,
            position=state[asset]["position"],
            trade=trade
        )


        if trade:
            log_trade(trade, signal, state[asset]["position"])

    if drawdown_exceeded(state):
        print("Drawdown limit hit. Stopping.")
        break

    print("-" * 80)
    time.sleep(LOOP_INTERVAL)

[17:37:53] BTC | Mid: 86740.50 | SMA: 86740.50 | EMA: 86740.50 | Vol_RSI: NA | Vol: 0.00 | Signal: FLAT | Position: 0
[17:37:53] ETH | Mid: 2863.55 | SMA: 2863.55 | EMA: 2863.55 | Vol_RSI: NA | Vol: 118.45 | Signal: FLAT | Position: 0
--------------------------------------------------------------------------------
[17:38:54] BTC | Mid: 86715.50 | SMA: 86728.00 | EMA: 86740.00 | Vol_RSI: NA | Vol: 0.12 | Signal: FLAT | Position: 0
[17:38:55] ETH | Mid: 2861.15 | SMA: 2862.35 | EMA: 2863.50 | Vol_RSI: NA | Vol: 1.29 | Signal: FLAT | Position: 0
--------------------------------------------------------------------------------
[17:39:56] BTC | Mid: 86577.50 | SMA: 86677.83 | EMA: 86736.79 | Vol_RSI: NA | Vol: 1.53 | Signal: FLAT | Position: 0
[17:39:57] ETH | Mid: 2854.55 | SMA: 2859.75 | EMA: 2863.33 | Vol_RSI: NA | Vol: 40.08 | Signal: FLAT | Position: 0
--------------------------------------------------------------------------------
[17:40:58] BTC | Mid: 86554.50 | SMA: 86647.00 | EMA: 8

KeyboardInterrupt: 

In [65]:
from data.hyperliquid import fetch_orderbook

ob = fetch_orderbook("BTC")
ob


{'coin': 'BTC',
 'time': 1766028055216,
 'levels': [[{'px': '86441.0', 'sz': '2.176', 'n': 5},
   {'px': '86440.0', 'sz': '0.14895', 'n': 3},
   {'px': '86439.0', 'sz': '0.00013', 'n': 1},
   {'px': '86438.0', 'sz': '0.05797', 'n': 2},
   {'px': '86437.0', 'sz': '0.40521', 'n': 2},
   {'px': '86436.0', 'sz': '0.05797', 'n': 2},
   {'px': '86435.0', 'sz': '0.00013', 'n': 1},
   {'px': '86434.0', 'sz': '0.00013', 'n': 1},
   {'px': '86433.0', 'sz': '0.18301', 'n': 3},
   {'px': '86432.0', 'sz': '4.18742', 'n': 8},
   {'px': '86431.0', 'sz': '1.92082', 'n': 4},
   {'px': '86430.0', 'sz': '0.41387', 'n': 3},
   {'px': '86429.0', 'sz': '0.20066', 'n': 4},
   {'px': '86428.0', 'sz': '1.80368', 'n': 3},
   {'px': '86427.0', 'sz': '0.06811', 'n': 3},
   {'px': '86426.0', 'sz': '1.86894', 'n': 4},
   {'px': '86425.0', 'sz': '0.91031', 'n': 6},
   {'px': '86424.0', 'sz': '0.07186', 'n': 3},
   {'px': '86423.0', 'sz': '0.6599', 'n': 7},
   {'px': '86422.0', 'sz': '0.51784', 'n': 5}],
  [{'px': '8

In [66]:
type(ob), ob.keys()


(dict, dict_keys(['coin', 'time', 'levels']))

In [67]:
levels = ob["levels"]

type(levels), len(levels)


(list, 2)

In [68]:
levels


[[{'px': '86441.0', 'sz': '2.176', 'n': 5},
  {'px': '86440.0', 'sz': '0.14895', 'n': 3},
  {'px': '86439.0', 'sz': '0.00013', 'n': 1},
  {'px': '86438.0', 'sz': '0.05797', 'n': 2},
  {'px': '86437.0', 'sz': '0.40521', 'n': 2},
  {'px': '86436.0', 'sz': '0.05797', 'n': 2},
  {'px': '86435.0', 'sz': '0.00013', 'n': 1},
  {'px': '86434.0', 'sz': '0.00013', 'n': 1},
  {'px': '86433.0', 'sz': '0.18301', 'n': 3},
  {'px': '86432.0', 'sz': '4.18742', 'n': 8},
  {'px': '86431.0', 'sz': '1.92082', 'n': 4},
  {'px': '86430.0', 'sz': '0.41387', 'n': 3},
  {'px': '86429.0', 'sz': '0.20066', 'n': 4},
  {'px': '86428.0', 'sz': '1.80368', 'n': 3},
  {'px': '86427.0', 'sz': '0.06811', 'n': 3},
  {'px': '86426.0', 'sz': '1.86894', 'n': 4},
  {'px': '86425.0', 'sz': '0.91031', 'n': 6},
  {'px': '86424.0', 'sz': '0.07186', 'n': 3},
  {'px': '86423.0', 'sz': '0.6599', 'n': 7},
  {'px': '86422.0', 'sz': '0.51784', 'n': 5}],
 [{'px': '86442.0', 'sz': '18.05693', 'n': 51},
  {'px': '86443.0', 'sz': '2.53093

In [69]:
bids = levels[0]
asks = levels[1]

print("BIDS type:", type(bids))
print("ASKS type:", type(asks))

print("First bid:", bids[0])
print("First ask:", asks[0])


BIDS type: <class 'list'>
ASKS type: <class 'list'>
First bid: {'px': '86441.0', 'sz': '2.176', 'n': 5}
First ask: {'px': '86442.0', 'sz': '18.05693', 'n': 51}


In [70]:
type(bids[0]), bids[0]


(dict, {'px': '86441.0', 'sz': '2.176', 'n': 5})